# ROC Mini‑Project (Colab‑Ready)
End‑to‑end pipeline: QC → DR → Clustering → Metrics → Markers → Denoising → Integration → GO.
**Fill the TODOs marked with 🔧.**


In [ ]:
# Install packages (Colab)
%%bash
pip -q install scanpy==1.9.6 anndata umap-learn python-igraph leidenalg scikit-learn gseapy bbknn harmonypy magic-impute mnnpy


In [ ]:
import scanpy as sc, anndata as ad
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import adjusted_rand_score
from pathlib import Path
import warnings; warnings.filterwarnings('ignore')
sc.settings.verbosity = 2
sc.set_figure_params(figsize=(4,4), dpi=120)
PROJECT_DIR = Path('..').resolve()
RESULTS = PROJECT_DIR / 'results'
FIGS = RESULTS / 'figs'
FIGS.mkdir(parents=True, exist_ok=True)
print('Results folder:', FIGS)


## 1) Load Data
🔧 Provide the path to your `.h5ad` file or assemble from 10x mtx files.


In [ ]:
adata_path = '../data/counts.h5ad'  # 🔧 change if needed
adata = sc.read_h5ad(adata_path)
adata.var_names_make_unique()
adata.obs_names_make_unique()
adata


## 2) QC & Normalization


In [ ]:
adata.var['mt'] = adata.var_names.str.upper().str.startswith(('MT-','MT_'))
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], inplace=True)
min_genes, max_genes = 200, 6000
max_mito = 0.15
sc.pp.filter_cells(adata, min_genes=min_genes)
adata = adata[adata.obs['pct_counts_mt'] < (max_mito*100)].copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=3000, flavor='seurat_v3')
adata = adata[:, adata.var.highly_variable].copy()
adata


## 3) PCA → Neighbors → UMAP


In [ ]:
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, n_comps=50, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=30)
sc.tl.umap(adata)
sc.pl.umap(adata, color=['n_genes_by_counts','total_counts','pct_counts_mt'])


## 4) Clustering (≥ 2 methods)


In [ ]:
sc.tl.leiden(adata, resolution=1.0, key_added='leiden_1.0')
sc.pl.umap(adata, color=['leiden_1.0'], legend_loc='on data')
from sklearn.cluster import KMeans
km = KMeans(n_clusters=12, random_state=0)
adata.obs['kmeans'] = km.fit_predict(adata.obsm['X_pca']).astype(str)
sc.pl.umap(adata, color=['kmeans'], legend_loc='on data')
adata.write('../results/adata_processed.h5ad')


## 5) Metrics


In [ ]:
from src.utils import compute_ari, compute_silhouette, purity_score
import pandas as pd
metrics = {}
embedding = adata.obsm['X_umap']
for key in ['leiden_1.0','kmeans']:
    metrics[f'silhouette_{key}'] = compute_silhouette(embedding, adata.obs[key])
    if 'cell_type' in adata.obs:
        metrics[f'ari_{key}'] = compute_ari(adata.obs['cell_type'], adata.obs[key])
        metrics[f'purity_{key}'] = purity_score(adata.obs['cell_type'], adata.obs[key])
pd.Series(metrics).to_csv('../results/metrics.csv')
metrics


## 6) Marker Selection (Wilcoxon & Logistic Regression)


In [ ]:
target_key = 'leiden_1.0'
sc.tl.rank_genes_groups(adata, groupby=target_key, method='wilcoxon', n_genes=200)
df_w = sc.get.rank_genes_groups_df(adata, group=None)
df_w.to_csv('../results/markers_wilcoxon.csv', index=False)
sc.tl.rank_genes_groups(adata, groupby=target_key, method='logreg', n_genes=200)
df_l = sc.get.rank_genes_groups_df(adata, group=None)
df_l.to_csv('../results/markers_logreg.csv', index=False)


## 7) Identify ROC cluster via overlap with Supplementary Table 3


In [ ]:
from src.utils import overlap_stats
ref_path = '../data/supp_table3_genes.txt'  # one gene per line
try:
    ref_genes = [g.strip() for g in open(ref_path).read().splitlines() if g.strip()]
except FileNotFoundError:
    ref_genes = []
    print('🔧 Add supp_table3_genes.txt under data/')
def top_genes_per_cluster(df, topn=30):
    out = {}
    for g, sub in df.groupby('group'):
        out[g] = list(sub.sort_values('scores', ascending=False).head(topn)['names'])
    return out
markers = top_genes_per_cluster(df_w, topn=30)
overlaps = []
for cl, genes in markers.items():
    stats = overlap_stats(genes, ref_genes)
    stats['cluster'] = cl
    overlaps.append(stats)
import pandas as pd
overlaps = pd.DataFrame(overlaps).sort_values('jaccard', ascending=False)
overlaps.to_csv('../results/roc_markers_overlap.csv', index=False)
overlaps.head()


## 8) Denoising: MAGIC (and optional MNN/DCA)


In [ ]:
import scanpy.external as sce
adata_magic = adata.copy()
sce.pp.magic(adata_magic)
sc.tl.pca(adata_magic, n_comps=50)
sc.pp.neighbors(adata_magic, n_neighbors=15, n_pcs=30)
sc.tl.umap(adata_magic)
sc.tl.leiden(adata_magic, resolution=1.0, key_added='leiden_magic')
from src.utils import compute_silhouette
sil_magic = compute_silhouette(adata_magic.obsm['X_umap'], adata_magic.obs['leiden_magic'])
pd.Series({'silhouette_leiden_magic': sil_magic}).to_csv('../results/metrics_magic.csv')
sil_magic


## 9) Batch Integration over time: BBKNN & Harmony (requires `timepoint` column)


In [ ]:
if 'timepoint' in adata.obs:
    # BBKNN
    adata_bbk = adata.copy()
    sc.pp.neighbors(adata_bbk, n_neighbors=15, n_pcs=30)
    sce.pp.bbknn(adata_bbk, batch_key='timepoint')
    sc.tl.umap(adata_bbk)
    sc.tl.leiden(adata_bbk, key_added='leiden_bbk')
    # Harmony
    adata_har = adata.copy()
    import harmonypy as hm
    sc.tl.pca(adata_har, n_comps=50)
    ho = hm.run_harmony(adata_har.obsm['X_pca'], adata_har.obs, 'timepoint')
    adata_har.obsm['X_pca_harmony'] = ho.Z_corr.T
    sc.pp.neighbors(adata_har, use_rep='X_pca_harmony')
    sc.tl.umap(adata_har)
    sc.tl.leiden(adata_har, key_added='leiden_harmony')
else:
    print('🔧 Add a timepoint column to adata.obs to run integration.')


## 10) GO Enrichment (Enrichr via GSEApy)


In [ ]:
import gseapy as gp
if len(overlaps)>0:
    roc_cluster = str(overlaps.iloc[0]['cluster'])
    roc_genes = markers[roc_cluster]
    enr = gp.enrichr(gene_list=roc_genes, gene_sets=['GO_Biological_Process_2021'], organism='Xenopus laevis', outdir='../results/go_enrichr', cutoff=0.5)
    enr.results.head()
else:
    print('🔧 Provide Supplementary Table 3 to run GO.')


In [ ]:
print('Pipeline complete. Saved outputs under ../results/.')
